In [4]:
from dash import Dash, html, dcc, callback, Output, Input, State
from pandas import Timestamp, date_range, Timedelta, NA
from dash.exceptions import PreventUpdate
from pysnmp.hlapi.v3arch.asyncio import *
import plotly.graph_objects as go
import asyncio

HOST_IP = '127.0.0.1'
STRING = 'public'

#total seconds between calls to determine data rate
ttl_secs = 0
#How long is the time window for the graph
graph_window = 90
intf_ID = '0' 
div_title = "Rx and Tx Data"

tx_ = []
rx_ = []
tx_prev = False
rx_prev = False
dates =  date_range(start=Timestamp.now()-Timedelta(seconds=graph_window), periods=graph_window, freq ='1s').to_list()
for i in range(len(dates)):
    tx_.append(NA)
    rx_.append(NA)

async def get_oid(host,string,oid,snmpEng):
    results = [];
    iterator = await getCmd(
        snmpEng,
        CommunityData(string, mpModel=1),
        await UdpTransportTarget.create((host, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = iterator
    if errorIndication:
        print(errorIndication)
    elif errorStatus:
        print(
            "{} at {}".format(
                errorStatus.prettyPrint(),
                errorIndex and varBinds[int(errorIndex) - 1][0] or "?",
            )
        )
    else:
        results =  varBinds
    return results

async def get_oids(host,string,oidData):
    snmpDispatcher = SnmpEngine()
    results = []
    first_oid = ''
    while True:
        errorIndication, errorStatus, errorIndex, varBindTable = await bulkCmd(
            snmpDispatcher,
            CommunityData(string, mpModel=1),
            await UdpTransportTarget.create((host, 161)),
            ContextData(),
            0,50,
            *oidData,
            lookupMib=False,
            lexicographicMode=True
        )
        if errorIndication:
            print(f"ERROR INDICATION :{errorIndication}")
            break
        elif errorStatus:
            print(
                f"ERROR STATUS {errorStatus.prettyPrint()} at {varBinds[int(errorIndex) - 1][0] if errorIndex else '?'}"
            )
        else:
            for varBind in varBindTable:
                if first_oid == varBind[0]:
                    break
                results.append({'oid' : f"{varBind[0]}", 'value': f"{varBind[1]}"})
        if first_oid == varBindTable[0][0]:
            break
        else:
            first_oid = varBindTable[0][0]
    return results

def draw_graph(rx,tx,date_list):
    global ttl_secs
    global div_title
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date_list, y=rx,
                        mode='lines',
                        #line_shape='spline',
                        name='Rx Bps'))
    fig.add_trace(go.Scatter(x=date_list, y=tx,
                        mode='lines',
                        #line_shape='spline',
                        name='Tx Bps'))
    fig.update_layout(
        title=f'<b>{div_title}</b><br>{Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")} - Update Rate: {int(ttl_secs)} seconds',
        title_x= 0.5,
        yaxis_title="Data Rate (BitsPerSecond)",
        paper_bgcolor='#404040',
        plot_bgcolor='#d3d3d3',
        font=dict(family="Courier New, monospace",color='white')
    )
    return fig

async def main(intID):
    global dates
    global rx_
    global tx_
    global tx_prev
    global rx_prev
    global ttl_secs
    global STRING
    snmpEngine = SnmpEngine()
    res = await asyncio.gather(
           get_oid(HOST_IP,STRING,f'1.3.6.1.2.1.2.2.1.10.{intf_ID}',snmpEngine),
           get_oid(HOST_IP,STRING,f'1.3.6.1.2.1.2.2.1.16.{intf_ID}',snmpEngine)
        )
    now = Timestamp.now()

    if len(res[0]) <=0 or len(res[1]) <=0:
        raise PreventUpdate
    else:
        rx_oct = int(res[0][0][1]) * 8
        tx_oct = int(res[1][0][1]) * 8
        if tx_prev == False:
            tx_prev = tx_oct
            rx_prev = rx_oct
            return draw_graph(rx_,tx_,dates)
        if tx_prev == tx_oct and rx_prev == rx_oct:
            return draw_graph(rx_,tx_,dates)
        tx_tmp = tx_oct
        rx_tmp = rx_oct
        tx_oct = int(tx_oct - tx_prev)
        rx_oct = int(rx_oct - rx_prev)

        tx_prev = tx_tmp
        rx_prev = rx_tmp
        ttl_secs = (now-dates[-1]).total_seconds()
        dates.append(now)
        tx_.append(tx_oct/ttl_secs)
        rx_.append(rx_oct/ttl_secs)
        if len(dates) > graph_window or len(tx_) > graph_window or len(rx_) > graph_window:
            dates = dates[-graph_window:]
            tx_ = tx_[-graph_window:]
            rx_ = rx_[-graph_window:]
        return draw_graph(rx_,tx_,dates)

oper_status = await get_oids(HOST_IP,STRING,[ObjectType(ObjectIdentity('1.3.6.1.2.1.2.2.1.8'))])
INTERFACES = []
idxList = []
for i in oper_status:
    if i['value'] == '1':
        oidIdx = f"{i['oid']}".rsplit('.',1)
        INTERFACES.append({'value': oidIdx[1]})
        idxList.append(oidIdx[1])
#'1.3.6.1.2.1.31.1.1.1.1'
#[ObjectType(ObjectIdentity("IF-MIB", "ifName"))]
intf_data = await get_oids(HOST_IP,STRING,[ObjectType(ObjectIdentity("1.3.6.1.2.1.31.1.1.1.1"))])
for idx, i in enumerate(intf_data):
    oidIdx = f"{i['oid']}".rsplit('.',1)
    if oidIdx[1] in idxList:
        if oidIdx[0] == '1.3.6.1.2.1.31.1.1.1.1':
            for j, d in enumerate(INTERFACES):
                if d['value'] == oidIdx[1]:
                    INTERFACES[j] = { 'value' : oidIdx[1], 'label' : f"{i['value']}" }

intf_data = await get_oids(HOST_IP,STRING,[ObjectType(ObjectIdentity('1.3.6.1.2.1.31.1.1.1.18'))])
for i in intf_data:
    oidIdx = f"{i['oid']}".rsplit('.',1)
    if oidIdx[1] in idxList:
        if oidIdx[0] == '1.3.6.1.2.1.31.1.1.1.18':
            for k, d in enumerate(INTERFACES):
                if d['value'] == oidIdx[1]:
                    old_data = d
                    INTERFACES[k] = { 'value' : d['value'] , 'label' : f"{INTERFACES[k]['label']} - {i['value']}" }

app = Dash(__name__,
           prevent_initial_callbacks='initial_duplicate')

app.layout = html.Div([
    dcc.Dropdown(id='int_select',options=INTERFACES,value=INTERFACES[0]['value'],clearable=False),
    dcc.Checklist(id='bRun', options=['Active'],value=['Active']),
    html.Div([
        dcc.Graph(id = 'live-graph', 
                    figure={
                        'layout': {
                            'title': f'<b>{div_title}</b><br>{Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")} - Update Rate: N/A',
                            'title_x' : 0.5,
                            'yaxis_title' : "Data Rate (BitsPerSecond)",
                            'plot_bgcolor': '#d3d3d3',
                            'paper_bgcolor': '#404040',
                            'font': {
                                'family': "Courier New, monospace",
                                'color': 'white'
                            }
                        }
                    }),

        dcc.Interval(
            id = 'graph-update',
            interval = 1000,
            n_intervals = 0
        )
    ])
])

@callback(
    Output('live-graph', 'figure'),
    Input('graph-update', 'n_intervals'),
    Input('bRun', 'value')
)

def update_graph(n,b):
    global intf_ID
    if 'Active' in b:
        return asyncio.run(main(intf_ID))
    else:
        raise PreventUpdate()

@callback(
    Output('live-graph', 'figure',allow_duplicate=True),
    Input("int_select",  "value"),
    State("int_select",  "options"),
    prevent_initial_call=True
)
def question(v,o):
    global intf_ID
    intf_ID = v
    global rx_
    global tx_
    global tx_prev
    global rx_prev
    global div_title
    tx_ = []
    rx_ = []
    tx_prev = False
    rx_prev = False
    label = [x['label'] for x in o if x['value'] == v]
    dates =  date_range(start = Timestamp.now()- Timedelta(seconds=graph_window), periods=graph_window, freq ='1s').to_list()
    for i in range(len(dates)):
       tx_.append(NA)
       rx_.append(NA)
    div_title = f"Rx and Tx Data - {label[0]}"
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=rx_,
                        mode='lines',
                        name='Rx Bps'))
    fig.add_trace(go.Scatter(x=dates, y=tx_,
                        mode='lines',
                        name='Tx Bps'))
    fig.update_layout(
        title=f'<b>{div_title}</b><br>{Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")} - Update Rate: N/A',
        title_x= 0.5,
        yaxis_title="Data Rate (BitsPerSecond)",
        paper_bgcolor='#404040',
        plot_bgcolor='#d3d3d3',
        font=dict(family="Courier New, monospace",color='white')
    )    
    return fig

if __name__ == '__main__':
    app.run(debug=True)